In [9]:
import os
import random
from collections import deque
import time
import math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.models import Model, load_model

In [2]:
path_server = './Vehicle-Routing-2/data/'

In [3]:
class VehicleRouterEnvironment:

    def __init__(self, path=path_server):
        self.path = path
        files = os.listdir(self.path)
        alldata = pd.DataFrame(index=None, columns=range(6))
        for filename in files:
            with open(os.path.join(self.path, filename)) as f:
                data = f.readlines()
                numdata = data[9:]
                file_data = pd.DataFrame(index=[int(line.split()[0]) for line in numdata],
                                         columns=range(6),
                                         data=[line.split()[1:] for line in numdata])  # start time at 0
                alldata = pd.concat([alldata, file_data], axis=0)

        alldata = alldata.astype("float")
        dist_scale = alldata.loc[:, [0, 1]].max().max()
        time_scale = alldata.loc[:, [3, 4]].max().max()
        self._timescale = time_scale
        self._distscale = dist_scale
        self._speedscaler = 1e-1 # bigger = vehicle moves slower

        self.scaler = np.array([dist_scale, dist_scale, 1, time_scale, time_scale])
        self._setup(problem_file_index=0, scaler=self.scaler)

    def _setup(self, problem_file_index=0, scaler=None):

        if scaler is None:
            scaler = np.array([1, 1, 1, 1, 1])

        files = os.listdir(self.path)
        self.problem_file = files[problem_file_index % len(files)]

        # read the txt file and load data into a dataframe
        with open(os.path.join(self.path, self.problem_file)) as datafile:
            data = datafile.readlines()
            headers = data[7]
            headers = headers.replace("CUST NO.", "CUST_NO.")
            headers = headers.replace("READY TIME", "READY_TIME")
            headers = headers.replace("DUE DATE", "DUE_DATE")
            numdata = data[9:]
            self.data = pd.DataFrame(index=[int(line.split()[0]) for line in numdata],
                                     columns=headers.split()[1:-1],
                                     data=[line.split()[1:] for line in numdata])  # start time at 0
            self.data = self.data.astype("float")
            self.service_time = self.data["SERVICE"][5]/self._timescale # 5 is random, 0 is always depot
            self.data.drop('SERVICE', axis=1, inplace=True)
            self.data = self.data.divide(scaler)
            self.data['COMPLETED?'] = 0  # keep track of who has been serviced
            self.data.loc[0, 'COMPLETED?'] = 1  # by default we say the depot has been serviced

        # define the state space and action space
        # state is the self.data dataframe + current time + current X coord + current Y coord
        self.time = 0  # start clock at 0
        self.max_time = self.data["DUE_DATE"].max() * 2

        self.XCOORD = self.data["XCOORD."][0] # depot is different at each file
        self.YCOORD = self.data["YCOORD."][0]

        self.action_space = self.data.index.tolist()  # action space is the customer numbers (ie: which customer to visit next)

        self._max_episode_steps = 1  # ? figure this is useful + is also in the framework for assignment 4

    def reset(self, episode_num):
        # refreshes the system to use a new problem file
        self._setup(problem_file_index=episode_num, scaler=self.scaler)
        return None

    def state(self):
        # state is 1) dist to customer 0, 2) customer times minus current time
        state = self.data.copy()
        state.drop('XCOORD.', axis=1, inplace=True)
        state.rename(columns={"YCOORD.": "DISTANCE"}, inplace=True)
        state.loc[:, ["READY_TIME", "DUE_DATE"]] = state.loc[:, ["READY_TIME", "DUE_DATE"]] - self.time
        state.loc[:, ["DISTANCE"]] = np.sqrt((self.data.loc[:, "XCOORD."] - self.XCOORD).pow(2) + \
                                             (self.data.loc[:, "YCOORD."] - self.YCOORD).pow(2))
        return state.values.flatten()

    def step(self, action):
        # returns next_state, reward, done, _

        # update the self.data (the state)
        prior_time, prior_Xcoord, prior_Ycoord = self.time, self.XCOORD, self.YCOORD
        next_Xcoord, next_Ycoord = self.data.loc[action][
            ['XCOORD.', 'YCOORD.']]  # next coords are defined by the next customer
        move_time = np.sqrt((prior_Xcoord - next_Xcoord) ** 2 + (prior_Ycoord - next_Ycoord) ** 2)
        move_time = move_time * self._speedscaler * self._distscale / self._timescale
        previously_completed = int(self.data.loc[action]['COMPLETED?'])
        ready_time = self.data.loc[action]['READY_TIME']
        due_date = self.data.loc[action]['DUE_DATE']

        if move_time == 0.:
            move_time = 1 / self._timescale

        # only include service time if we haven't already serviced this customer and if we visited after the ready time
        if previously_completed == 0 and prior_time + move_time >= ready_time:
            next_time = prior_time + move_time + self.service_time
            self.data.loc[action, "COMPLETED?"] = 1
        else:
            next_time = prior_time + move_time

        # # work out rewards
        # # let reward be +1 if current delivery completed on time AND we arrived to them at least after the ready time
        # # -1 reward for every delivery across the system that is late
        #reward = 10*int((due_date >= next_time) & (previously_completed == 0) & (prior_time + move_time >= ready_time)) \
        #         - np.sum((self.data.loc[:, 'COMPLETED?'] == 0) & (self.data.loc[:, 'DUE_DATE'] < next_time) * self.data.loc[:, "DUE_DATE"] <= next_time) \
        #         - (previously_completed == 1) # subtract if revisiting somewhere

        reward = int((due_date >= next_time) & (previously_completed == 0) & (prior_time + move_time >= ready_time)) \
                - np.sum((self.data.loc[:, 'COMPLETED?'] == 0) & (self.data.loc[:, 'DUE_DATE'] < next_time)) / 100

        reward = float(reward)#/100 # normalise reward

        self.time = next_time
        self.XCOORD = next_Xcoord
        self.YCOORD = next_Ycoord

        # now set the newly serviced customer's COMPLETED? to 1 (True)
        done = 0 not in self.data.loc[:, 'COMPLETED?'].tolist()  # done is True if everyone has been serviced

        if next_time > self.max_time:   # done also True if max time reached
            done = True if next_time > self.max_time else done # done also True if max time reached

        # print('new state following action: \n{}'.format(self.data))
        # print('action: {}'.format(action))
        # print('time: {}'.format(self.time))
        # print('reward: {}'.format(reward))
        # print('done? {}'.format(done))

        return self.state(), reward, done, None

In [4]:
def OurModel(input_shape, action_space):
    X_input = Input(input_shape)
    X = Dense(1028, input_shape=input_shape, activation="relu", kernel_initializer='he_uniform')(X_input)
    X = Dropout(0.25)(X)
    X = Dense(512, activation="relu", kernel_initializer='he_uniform')(X)
    X = Dense(256, activation="relu", kernel_initializer='he_uniform')(X)
    X = Dropout(0.25)(X)
    # X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X)
    X = Dense(action_space, activation="linear", kernel_initializer='he_uniform')(X)

    model = Model(inputs = X_input, outputs = X, name='DQN_model')
    model.compile(loss="mse", optimizer=RMSprop(lr=0.0025, rho=0.95, epsilon=0.01), metrics=["accuracy"])

    model.summary()
    return model

In [7]:
class DQNAgent:
    def __init__(self):
        self.env = VehicleRouterEnvironment()
        self.state_size = len(self.env.state())
        self.action_size = len(self.env.action_space)
        self.EPISODES = 1000
        self.memory = deque(maxlen=10000)
        
        self.gamma = 0.99   # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.001
        self.epsilon_decay = 0.999
        self.batch_size = 64
        self.train_start = 1000

        # create main model
        self.model = OurModel(input_shape=(self.state_size,), action_space = self.action_size)

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if len(state.shape) == 1: # catch for dimensionality issues
            state = np.array(state).reshape([1, len(state)])

        if np.random.uniform(0,1) < self.epsilon:
            return np.random.choice(self.env.action_space)
        else:
            q_out = self.model.predict(state)
            action = np.argmax(q_out)
            return action

    def replay(self):
        if len(self.memory) < self.train_start:
            return

        # Randomly sample minibatch from the memory
        t0 = time.time()
        minibatch = random.sample(self.memory, min(len(self.memory), self.batch_size))
        #print('1: ', time.time() - t0)

        state = np.zeros((self.batch_size, self.state_size))
        next_state = np.zeros((self.batch_size, self.state_size))
        action, reward, done = [], [], []

        # assign data into state, next_state, action, reward and done from minibatch
        t0 = time.time()
        for i in range(self.batch_size):
            ministate, miniaction, minireward, mininextstate, minidone = minibatch[i]
            state[i] = ministate.reshape([1, self.state_size])
            next_state[i] = mininextstate
            action.append(self.act(state[i]))
            reward.append(minireward)
            done.append(minidone)
        action = np.array(action)
        reward = np.array(reward)
        #print('2: ', time.time() - t0)

        t0 = time.time()
        flat_target = self.model.predict(state).flatten()
        Q_target_next = np.max(self.model.predict(next_state), axis=1)
        action_indices = action + np.arange(self.batch_size) * self.action_size # provides index in flattened array of where this action maps to
        flat_target[action_indices] = reward + self.gamma * Q_target_next.flatten()
        target = flat_target.reshape([self.batch_size, self.action_size]) # pull back into shape
        #print('3: ', time.time() - t0)

        #target = np.zeros((self.batch_size, self.action_size))

        ## compute value function of current(call it target) and value function of next state(call it target_next)
        #for i in range(self.batch_size):
        #    a = action[i]
        #    if done[i]:
        #        target[i] = self.model.predict(state[i].reshape([1, len(state[i])]))
        #        target[i, a] = reward[i]
        #    else:
        #        Q_target_next = np.max(self.model.predict(next_state[i].reshape([1, len(next_state[i])])))
        #        target[i] = self.model.predict(state[i].reshape([1, len(state[i])]))
        #        target[i, a] = reward[i] + self.gamma * Q_target_next

        t0 = time.time()
        self.model.fit(state, target, batch_size=self.batch_size, verbose=0, epochs=1)
        #print('4: ', time.time() - t0)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model = load_model(name)

    def save(self, name):
        self.model.save(name)
            
    def training(self):
        scores = []
        stops = []

        for e in range(self.EPISODES):
            self.env.reset(e)
            state = self.env.state()

            done = False
            i = 0

            actions = []
            total_reward = 0
            while not done:
                action = self.act(state)
                actions.append(action)
                next_state, reward, done, _ = self.env.step(action)
                total_reward += reward

                self.remember(state, action, reward, next_state, done)

                state = next_state
                i += 1
                if done:
                    # dateTimeObj = datetime.now()
                    # timestampStr = dateTimeObj.strftime("%H:%M:%S")
                    print("episode: {}/{}, stops_made: {}, epsilon: {:.2}, total summed reward: {}".format(e+1, self.EPISODES, i, self.epsilon, total_reward))
                    scores.append(total_reward)
                    stops.append(i)

                #if math.log(i, 10) % 1 > math.log(i+1, 10) % 1: # only replay when the num episodes has crossed an order of magnitude
                    # this speeds things up
                    # self.replay()
                if i % 10 == 0:
                    self.replay()

        return scores, stops

In [10]:
agent = DQNAgent()
scores,stops = agent.training()

Model: "DQN_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 505)]             0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               129536    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_6 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 101)               13

episode: 81/1000, stops_made: 136, epsilon: 0.25, total summed reward: -62.530000000000015
episode: 82/1000, stops_made: 74, epsilon: 0.25, total summed reward: -33.99999999999999
episode: 83/1000, stops_made: 105, epsilon: 0.25, total summed reward: -65.66000000000001
episode: 84/1000, stops_made: 539, epsilon: 0.24, total summed reward: -100.30000000000015
episode: 85/1000, stops_made: 105, epsilon: 0.23, total summed reward: -61.50000000000001
episode: 86/1000, stops_made: 121, epsilon: 0.23, total summed reward: -49.12000000000002
episode: 87/1000, stops_made: 371, epsilon: 0.22, total summed reward: -112.76999999999981
episode: 88/1000, stops_made: 83, epsilon: 0.22, total summed reward: -29.040000000000003
episode: 89/1000, stops_made: 114, epsilon: 0.22, total summed reward: -50.65000000000002
episode: 90/1000, stops_made: 94, epsilon: 0.22, total summed reward: -38.37
episode: 91/1000, stops_made: 104, epsilon: 0.21, total summed reward: -51.860000000000014
episode: 92/1000, st

KeyboardInterrupt: 

In [48]:
agent.env.problem_file

'r202.txt'

In [ ]:
episodes = np.arange(len(scores))
plt.plot(episodes, scores)
plt.xlabel('Episodes')
plt.ylabel('Score')
plt.show()

In [ ]:
plt.plot(episodes, stops)
plt.xlabel('Episodes')
plt.ylabel('Stops')
plt.show()